## Restore data
The purpose of this notebook is revert changes that were done during one of the previous batch modifications
Data is obtained from the backup data created during recent modification.

In [ ]:
# setting notebook source
with open('./config_path.py') as f:
    exec(f.read())

In [ ]:
from config import secrets, settings
from zoho_api import *
import pandas as pd
import copy
from dateutil import parser
from datetime import date, time

In [ ]:
backup_data = {}
response_data = {}

In [ ]:
refresh_token()

In [ ]:
SLAs = {}
SLAs['SupportME Premium'] = '10977000006774075'
SLAs['SupportME Comfort'] = '10977000006804083'
SLAs['SupportME ComfortPlus'] = '10977000006804083'
SLAs['SupportME Standard'] = '10977000006804131'
SLAs[None] = ''
SLAs

In [ ]:
SupportHours = {}
SupportHours['SupportME Premium'] = 204
SupportHours['SupportME Comfort'] = 96
SupportHours['SupportME ComfortPlus'] = 8
SupportHours['SupportME Standard'] = 36
SupportHours[None] = None
SupportHours

## Application

In [ ]:
field_list = ['id'
              , 'accountName'
              , 'cf_account_id'
              , 'website'
              , 'cf_account_id'
              , 'cf_support_package'
              , 'cf_support_subsription_expiry_date'
              , 'associatedSLAIds'
              , 'cf_max_support_time'
              , 'cf_picklist_1'  # Customer type
              , 'cf_loyality_program'
             ]

In [ ]:
accounts = load_backup_from_file(filename='../backups/response_2025-04-22.json')

In [ ]:
df_accounts = pd.json_normalize(accounts.values())

In [ ]:
columns_to_keep = ['id', 'accountName', 'website','associatedSLAIds', 'cf.cf_account_id', 'cf.cf_max_support_time', 'cf.cf_picklist_1', 'cf.cf_support_package','cf.cf_support_subsription_expiry_date',
       'cf.cf_loyality_program']

In [ ]:
df_accounts = df_accounts[columns_to_keep]

In [ ]:
# drop not associated accounts
df_accounts.dropna(subset=[ 'cf.cf_account_id','cf.cf_picklist_1'], inplace=True)

In [ ]:
# filter out non-SI customers
df_accounts = df_accounts[df_accounts['cf.cf_picklist_1'].str.contains('SI')]

In [ ]:
df_accounts[df_accounts['cf.cf_loyality_program'].isin(['SI Registered', 'SI Member','SI Elite'])]['cf.cf_support_package'].value_counts()

In [ ]:
update_df = df_accounts[df_accounts['cf.cf_loyality_program'].isin(['SI Registered', 'SI Member','SI Elite'])]

In [ ]:
accounts_updated = copy.deepcopy(accounts)
modifications = {}
for account in update_df.iterrows():
    support_package = ''
    
    # validation of expiry date
    if account[1]['cf.cf_support_package'] == 'Standard':
        support_package = 'SupportME Standard'
    elif account[1]['cf.cf_support_package'] == 'Comfort':
        support_package = 'SupportME Comfort'
    elif account[1]['cf.cf_support_package'] == 'Premium':
        support_package = 'SupportME Premium'
    else:
        raise Exception()
        
    my_dict = dict()
    # if any modification have been applied on data, prepare setup
    if support_package != '':
        
        my_dict['cf'] = {}
        my_dict['cf']['cf_support_package'] = support_package
        my_dict['cf']['cf_support_subsription_expiry_date'] = '2025-06-30'
        my_dict['cf']['cf_max_support_time'] = SupportHours[support_package]       
        modifications[ account[1].pop('id') ] =copy.deepcopy(my_dict)

In [ ]:
modifications

In [ ]:
processedIds = set()
missingIds = set()

In [ ]:
refresh_token()

for index, record in enumerate(modifications.items()):
    if record[0] in processedIds:
        continue
    account = backup_updated_fields(record[0], record[1], backup_data, response_data)
    if account is None:
        missingIds.add(record[0])
    else:
        try:
            account, response = update_account(record[0], record[1])
            if account is None or response.status_code != 200:
                missingIds.add(record[0])
                print(f"\r{index + 1}) Error for updating account: {record[0]}. \t\t")
            else:
                support_package = record[1]['cf']['cf_support_package']
                print(f"{index + 1}) Data for {record[0]} have been updated.\t", end='')
                apply_sla(record[0], SLAs[support_package])
                processedIds.add(record[0])
        except Exception as e:   
            missingIds.add(record[0])
            print(f"\r{index + 1}) Error for executing script for account: {record[0]}: {e}")
print("Updating process have been finished")

In [ ]:
updated_fields = record[1]
today = date.today()
response_filename = "response_" +str(today) + "SI_restore.json"
backup_filename = "backup_" +str(today) + "SI_restore.json"
save_response_to_file(response_data = response_data,  filename=response_filename, updated_fields=updated_fields)
save_backup_to_file(backup_data, filename=backup_filename, updated_fields=updated_fields)